In [1]:
# the original version
import random

in_file = open('./DATA_MI.txt','r')
out_file = open('./DATA_MI_cost.txt','w')

line = in_file.readline()
while line:
    data = line.split(' ')
    if len(data) == 4 :# means this line is a road statement
        road_length = float(data[2])
        road_length = road_length * 10000
        data[2] = str(road_length)
    else:   # means this line is a point statement
        point_set = []
        for i in range(0,len(data)-1,2):
            distance = float(data[i+1])
            distance = distance * 10000
            data[i+1] = str(distance)
            point=[data[i],data[i+1],str(random.randint(10,90))]
            point_set.extend(point)
        data = point_set
        data.extend('\n')
    line = ' '.join(data)
    out_file.write(line)
    line = in_file.readline()

in_file.close()
out_file.close()

In [2]:
# updated version 2019/07/29
# assign cost for each poi
import random

in_file = open('./DATA_MI.txt','r')
out_file = open('./DATA_MI_cost.txt','w')
point_count = 0
road_count = 0
is_road = True
poi_num = 0

line = in_file.readline()
while line:
    data = line.split(' ')
    
    if len(data) <= 1:
        line  = in_file.readline()
        continue   # skip blank lines
    if is_road:
        road_count += 1
        poi_num = int(data[3].strip())
        if poi_num == 0:
            is_road = True # next line is still road
        else:
            is_road = False # next line is point description
    else:   # means this line is a point statement
        point_set = []
        for i in range(0,len(data)-1,2):
            point_count += 1
            point=[data[i],data[i+1],str(random.randint(10,90))]
            point_set.extend(point)
        data = point_set
        data.extend('\n')
        is_road = True # next line is road
    line = ' '.join(data)
    out_file.write(line)
    line = in_file.readline()

in_file.close()
out_file.close()
print('random poi cost append done. there are %d pois and %d roads.' % (point_count,road_count))

random poi cost append done. there are 22 pois and 595726 roads.


In [2]:
# updated version 2020/09/14
# assign cost for each poi with rating from foursquare dataset

foursquare_file=open('./foursuqareRating.txt','r')
in_file = open('./DATA_MI.txt','r')
out_file = open('./DATA_MI_cost.txt','w')
point_count = 0
road_count = 0
is_road = True
poi_num = 0

# obtain rating data from file
foursquare_file_content=foursquare_file.read()
rating_list=foursquare_file_content.split()
total_rating_count=len(rating_list)
print('the number of rating in foursquare file is:',total_rating_count)
rating_p=0

line = in_file.readline()
while line:
    data = line.split(' ')
    
    if len(data) <= 1:
        line  = in_file.readline()
        continue   # skip blank lines
    if is_road:
        road_count += 1
        poi_num = int(data[3].strip())
        data[2]=str(float(data[2])*1000)
        if poi_num == 0:
            is_road = True # next line is still road
        else:
            is_road = False # next line is point description
    else:   # means this line is a point statement
        point_set = []
        for i in range(0,len(data)-1,2):
            point_count += 1
            distance=float(data[i+1])
            distance*=1000
            point=[data[i],str(distance),rating_list[rating_p%total_rating_count]]
            rating_p+=1
            point_set.extend(point)
        data = point_set
        data.extend('\n')
        is_road = True # next line is road
    line = ' '.join(data)
    out_file.write(line)
    line = in_file.readline()

in_file.close()
out_file.close()
foursquare_file.close()
print('poi cost assigning done. there are %d pois and %d roads.' % (point_count,road_count))

the number of rating in foursquare file is: 84291
poi cost assigning done. there are 22 pois and 595726 roads.


In [3]:
len(total_points)

87635

In [2]:
#sampled version
"""
分成road statement 和 point statement两部分
road statement包含起止结点和道路长度，是不需要修改的
point statement包含若干个POI以及距离起始节点的距离，将提取出POI并进行采样
"""
import random
import pandas as pd
import numpy as np

in_file = open('./calmap.txt','r')
rate = 0.5
out_file = open('./calmap_cost_'+str(int(rate * 100))+'.txt','w')

total_points=[]
road_statements=[]
line = in_file.readline()
start_node = 0
end_node = 0
while line:
    data = line.split(' ')
    if len(data) == 4 :# means this line is a road statement
        road_length = float(data[2])
        road_length = road_length * 1000
        data[2] = str(road_length)
        start_node = data[0]
        end_node = data[1]
        road_statements.append(data)
    else:   # means this line is a point statement
        point_set = []
        for i in range(0,len(data)-1,2):
            distance = float(data[i+1])
            distance = distance * 1000
            data[i+1] = str(distance)
            category_ID_point = data[i] 
            point=[start_node,end_node,category_ID_point,data[i+1],str(random.randint(10,90))]
            total_points.append(point)
    line = in_file.readline()
print("read file done.")

df = pd.DataFrame(total_points,columns=['start_node','end_node','category_id','distance','cost'])
df_sampled=df.sample(frac=rate,random_state=1)
length = len(df_sampled.category_id.value_counts())
old = df_sampled.category_id.value_counts().index
new = [str(x) for x in range(length)]
df_sampled["category_id"].replace(dict(zip(old,new)),inplace = True)

print("sample done.new map contains ",length," categories.")

#write into file
for line in road_statements:
    point_num = int(line[3].strip())
    if point_num == 0:
        string_to_write = ' '.join(line)
        out_file.write(string_to_write)
        continue
    df_query =  df_sampled[(df_sampled.start_node == line[0]) & (df_sampled.end_node == line[1])]
    df_list = np.array(df_query).tolist()
    if len(df_list) == 0:
        line[3]='0\n'
        string_to_write = ' '.join(line)
        out_file.write(string_to_write)
        continue
    point_statement=''
    for item in df_list:
        item = item[2:5]
        tmp = ' '.join(item)+' '
        point_statement+= tmp
    point_statement+='\n'
    
    line[3]=str(len(df_list))+'\n'
    out_file.write(' '.join(line))
    out_file.write(point_statement)
print('write to file done.')
in_file.close()
out_file.close()

read file done.
sample done.new map contains  61  categories.
write to file done.


In [54]:
#playground
df_sampled

,start_node,end_node,category_id,distance,cost
22610,6478,6479,16,19.690800000000003,79
14139,4666,4667,25,3.22153,84
23502,6654,6655,4,21.610999999999997,28
79024,18761,18762,12,11.2303,54
73240,17784,17785,0,13.3008,58
...,...,...,...,...,...
84609,20340,20356,1,10.2166,60
34820,8426,8427,0,0.10045200000000001,63
38468,8876,8882,0,16.371299999999998,36
18061,5550,5551,21,1.82124,86


## check

In [4]:
in_file = open('./cal/calmap_cost_reduced_01.txt','r')
#out_file = open('./cal/calmap_cost_reduced_01.txt','w')

total_points=[]
road_statements=[]
line = in_file.readline()
start_node = 0
end_node = 0
while line:
    data = line.split(' ')
    if len(data) == 4 :# means this line is a road statement
        road_length = float(data[2])
        road_length = road_length * 1000
        data[2] = str(road_length)
        start_node = data[0]
        end_node = data[1]
        road_statements.append(data)
    else:   # means this line is a point statement
        point_set = []
        for i in range(0,len(data)-1,2):
            distance = float(data[i+1])
            distance = distance * 1000
            data[i+1] = str(distance)
            point=[start_node,end_node,data[i],data[i+1],str(random.randint(10,90))]
            total_points.append(point)
    line = in_file.readline()
    
df = pd.DataFrame(total_points,columns=['start_node','end_node','category_id','distance','cost'])

In [5]:
df

,start_node,end_node,category_id,distance,cost
0,3,4,48,9718.51,22
1,9,10,44,7762.630000000001,41
2,9,36,38,2723.77,85
3,9,36,8555,373.468,25
4,11,12,53,4584.82,13
5,11,12,1660,2505.53,43
6,17,18,36,7832.650000000001,44
7,18,19,54,7036.4800000000005,14
8,28,29,21,20654.9,21
9,33,34,54,24375.1,59
